In [ ]:
!wget -q --show-progress https://drive.google.com/drive/folders/1SLclsq_kKL27VgVjq2hr8JsT45kx1pEy

1SLclsq_kKL27VgVjq2     [ <=>                ] 167.67K  --.-KB/s    in 0.02s   


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#from google.colab import drive
#drive.flush_and_unmount()

In [ ]:
import cv2
import os

def extract_frames(video_path, output_folder, frame_rate=30):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_name = os.path.join(output_folder, f'frame_{count}.jpg')
            cv2.imwrite(frame_name, frame)
        count += 1
    cap.release()

In [ ]:
def detect_faces_and_eyes(image_path, face_cascade, eye_cascade):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    eyes = []
    for (x, y, w, h) in faces:
        face_roi = img[y:y+h, x:x+w]
        gray_face = gray[y:y+h, x:x+w]
        detected_eyes = eye_cascade.detectMultiScale(gray_face)
        for (ex, ey, ew, eh) in detected_eyes:
            eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
            eyes.append((eye_roi, (x+ex, y+ey, ew, eh)))  # Return eye image and bounding box
    return eyes


In [ ]:
def align_and_normalize_eyes(eyes, output_size=(224, 224)):
    normalized_eyes = []
    for eye in eyes:
        normalized_eye = cv2.resize(eye, output_size)
        normalized_eyes.append(normalized_eye)
    return normalized_eyes


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

def augment_eyes(eye_images):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.8, 1.2],
        zoom_range=0.2,
        horizontal_flip=True
    )
    augmented_images = []
    for eye_image in eye_images:
        eye_image = eye_image.reshape((1, ) + eye_image.shape)  # Reshape for Keras
        for batch in datagen.flow(eye_image, batch_size=1):
            augmented_images.append(batch[0])
            if len(augmented_images) >= 5:  # Limiting to 5 augmentations per image for example
                break
    return np.array(augmented_images)


In [ ]:
import glob

dataset_folder = '/content/drive/MyDrive/NeurIPS/Code/Dataset(LPW)/LPW/1'

def process_videos(dataset_folder):
    video_files = glob.glob(os.path.join(dataset_folder, '*.avi'))
    for video_file in video_files:
        video_name = os.path.basename(video_file).split('.')[0]
        frames_folder = os.path.join(dataset_folder, video_name, 'frames')
        extract_frames(video_file, frames_folder)

        frame_files = glob.glob(os.path.join(frames_folder, '*.jpg'))
        for frame_file in frame_files:

            eyes = detect_faces_and_eyes(frame_file, face_cascade, eye_cascade)
            if eyes:
                aligned_eyes = align_and_normalize_eyes(eyes)
                augmented_eyes = augment_eyes(aligned_eyes)
                # Save or use the augmented eye images
                for idx, eye_image in enumerate(augmented_eyes):
                    eye_image_path = os.path.join(dataset_folder, video_name, 'eyes', f'eye_{idx}.jpg')
                    if not os.path.exists(os.path.dirname(eye_image_path)):
                        os.makedirs(os.path.dirname(eye_image_path))
                    cv2.imwrite(eye_image_path, eye_image)

process_videos(dataset_folder)


NameError: name 'face_cascade' is not defined

In [ ]:
process_videos(dataset_folder)

In [ ]:
!pip install keras
!pip install pytube
!pip install yt-dlp


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 32.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.4/194.4 kB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 5.8 MB/s eta 0:00:00


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
import os
import pandas as pd
from pytube import YouTube
import yt_dlp

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# Read CSV file, trying different encodings
csv_file_path = '/content/drive/MyDrive/NeurIPS/Code/dataset_links_drunk.csv'

try:
    df = pd.read_csv(csv_file_path, encoding='utf-8')  # Try UTF-8 first
except UnicodeDecodeError:
    try:
        df = pd.read_csv(csv_file_path, encoding='latin-1')  # Try Latin-1 if UTF-8 fails
    except UnicodeDecodeError:
        df = pd.read_csv(csv_file_path, encoding='cp1252')  # Try cp1252 as a last resort

# Extract video links
print(df.head())
video_links = df['link'].tolist()

                                                link                    name  \
0  https://www.youtube.com/watch?v=x24jb5ww4lQ&li...  videoplayback (40).mp4   
1        https://www.youtube.com/watch?v=bvfEYtzq3fE  videoplayback (39).mp4   
2        https://www.youtube.com/watch?v=4QdSIQonzI4  videoplayback (38).mp4   
3        https://www.youtube.com/watch?v=H1KluhZpT1Y  videoplayback (37).mp4   
4        https://www.youtube.com/watch?v=SIH90qFVT04  videoplayback (36).mp4   

  Unnamed: 2  
0        NaN  
1        NaN  
2        NaN  
3        NaN  
4        NaN  


In [ ]:
def download_youtube_video(url, output_path):
    yt = YouTube(url)
    stream = yt.streams.filter(progressive=True, file_extension='mp4').first()
    stream.download(output_path=output_path)

# Create a folder for the videos
videos_folder = '/content/videos'
if not os.path.exists(videos_folder):
    os.makedirs(videos_folder)

# Download each video
for link in video_links:
    try:
        download_youtube_video(link, videos_folder)
    except Exception as e:
        print(f"Failed to download {link}: {e}")

Failed to download https://www.youtube.com/watch?v=yOyHVKgVH8A: yOyHVKgVH8A is unavailable
Failed to download https://www.youtube.com/watch?v=tVMk7atYjBc: tVMk7atYjBc is a private video
Failed to download https://www.youtube.com/watch?v=91UCO3ii8g8: 91UCO3ii8g8 is a private video
Failed to download https://www.youtube.com/watch?v=RZ5dj-Ozwm0: RZ5dj-Ozwm0 is age restricted, and can't be accessed without logging in.
Failed to download https://www.youtube.com/watch?v=0NbBjNiw4tk: 0NbBjNiw4tk is age restricted, and can't be accessed without logging in.
Failed to download https://www.youtube.com/watch?v=9XjS4I4oQDY: 9XjS4I4oQDY is age restricted, and can't be accessed without logging in.
Failed to download https://www.youtube.com/watch?v=z4QMvfMXY2c: z4QMvfMXY2c is unavailable
Failed to download nan: expected string or bytes-like object
Failed to download nan: expected string or bytes-like object
Failed to download https://www.youtube.com/watch?v=d1xb0_tT5SQ: d1xb0_tT5SQ is unavailable
Fail

In [ ]:
def extract_frames(video_path, output_folder, frame_rate=30):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    cap = cv2.VideoCapture(video_path)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        if count % frame_rate == 0:
            frame_name = os.path.join(output_folder, f'frame_{count}.jpg')
            cv2.imwrite(frame_name, frame)
        count += 1
    cap.release()

In [ ]:
def detect_faces_and_eyes(image_path, face_cascade, eye_cascade):
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    eyes = []
    for (x, y, w, h) in faces:
        face_roi = img[y:y+h, x:x+w]
        gray_face = gray[y:y+h, x:x+w]
        detected_eyes = eye_cascade.detectMultiScale(gray_face)
        for (ex, ey, ew, eh) in detected_eyes:
            eye_roi = face_roi[ey:ey+eh, ex:ex+ew]
            eyes.append((eye_roi, (x+ex, y+ey, ew, eh)))  # Return eye image and bounding box
    return eyes

In [ ]:
def detect_iris_and_pupil(eye):
    gray_eye = cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    gray_eye = cv2.medianBlur(gray_eye, 5)
    circles = cv2.HoughCircles(gray_eye, cv2.HOUGH_GRADIENT, dp=1, minDist=gray_eye.shape[0]/8,
                               param1=100, param2=30, minRadius=10, maxRadius=30)
    if circles is not None:
        circles = np.uint16(np.around(circles))
        for i in circles[0, :]:
            center = (i[0], i[1])  # center
            radius = i[2]  # radius
            cv2.circle(eye, center, radius, (0, 255, 0), 2)  # Draw the outer circle
            cv2.circle(eye, center, 2, (0, 0, 255), 3)  # Draw the center of the circle (pupil)
    return eye

In [ ]:
def align_and_normalize_eyes(eyes, output_size=(224, 224)):
    normalized_eyes = []
    for eye, _ in eyes:
        normalized_eye = cv2.resize(eye, output_size)
        normalized_eyes.append(normalized_eye)
    return normalized_eyes

In [ ]:
def augment_eyes(eye_images):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        brightness_range=[0.8, 1.2],
        zoom_range=0.2,
        horizontal_flip=True
    )
    augmented_images = []
    for eye_image in eye_images:
        eye_image = eye_image.reshape((1, ) + eye_image.shape)  # Reshape for Keras
        for batch in datagen.flow(eye_image, batch_size=1):
            augmented_images.append(batch[0])
            if len(augmented_images) >= 5:  # Limiting to 5 augmentations per image for example
                break
    return np.array(augmented_images)

In [ ]:
def draw_bounding_boxes(image, eyes):
    for _, (x, y, w, h) in eyes:
        cv2.rectangle(image, (x, y), (x+w, y+h), (255, 0, 0), 2)  # Draw bounding box around the eye
    return image

# Load Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_eye.xml')

In [ ]:
import glob

dataset_folder = '/content/drive/MyDrive/NeurIPS/Code/Dataset(LPW)/LPW/1'
def process_videos(dataset_folder):
    video_files = glob.glob(os.path.join(dataset_folder, '*.mp4'))
    for video_file in video_files:
        video_name = os.path.basename(video_file).split('.')[0]
        frames_folder = os.path.join(dataset_folder, video_name, 'frames')
        extract_frames(video_file, frames_folder)

        frame_files = glob.glob(os.path.join(frames_folder, '*.jpg'))
        for frame_file in frame_files:
            eyes = detect_faces_and_eyes(frame_file, face_cascade, eye_cascade)
            if eyes:
                aligned_eyes = align_and_normalize_eyes(eyes)
                augmented_eyes = augment_eyes(aligned_eyes)

                img = cv2.imread(frame_file)
                img_with_boxes = draw_bounding_boxes(img, eyes)

                for idx, eye_image in enumerate(augmented_eyes):
                    eye_image_path = os.path.join(dataset_folder, video_name, 'eyes', f'eye_{idx}.jpg')
                    if not os.path.exists(os.path.dirname(eye_image_path)):
                        os.makedirs(os.path.dirname(eye_image_path))
                    cv2.imwrite(eye_image_path, eye_image)

                # Save the frame with bounding boxes
                frame_with_boxes_path = os.path.join(dataset_folder, video_name, 'frames_with_boxes', os.path.basename(frame_file))
                if not os.path.exists(os.path.dirname(frame_with_boxes_path)):
                    os.makedirs(os.path.dirname(frame_with_boxes_path))
                cv2.imwrite(frame_with_boxes_path, img_with_boxes)

In [ ]:
process_videos(videos_folder)

In [ ]:
import shutil

source_root = '/content/videos/'
destination_directory = '/content/sober_eye_images'
os.makedirs(destination_directory, exist_ok=True)


In [ ]:
image_count = 0

for root, dirs, files in os.walk(source_root):
    for dir_name in dirs:
        if dir_name == 'eyes':
            eyes_folder_path = os.path.join(root, dir_name)
            for image_name in os.listdir(eyes_folder_path):
                image_path = os.path.join(eyes_folder_path, image_name)
                if os.path.isfile(image_path):
                    new_image_name = f'Sober{image_count}.jpg'
                    new_image_path = os.path.join(destination_directory, new_image_name)
                    shutil.copy(image_path, new_image_path)
                    image_count += 1

print(f"Total images copied: {image_count}")

Total images copied: 549
